In [95]:
# Aplicação completa:

In [159]:
import os
import json
import sys
from tqdm import tqdm

# Define o caminho para a pasta 'corpus'
pasta_corpus = r'C:\Users\tagsa\Downloads\corpus'

# Função para carregar e verificar os textos dos arquivos JSON
def carregar_textos_json(pasta):
    textos = []
    arquivos_validos = 0
    textos_encontrados = 0

    if not os.path.exists(pasta):
        print(f"Pasta não encontrada: {pasta}")
        return '', [], 0, 0

    arquivos = [arq for arq in os.listdir(pasta) if arq.endswith('.json')]
    print(f"Número de arquivos JSON encontrados: {len(arquivos)}")

    for arquivo in tqdm(arquivos, desc="Carregando arquivos JSON", unit="arquivo"):
        caminho = os.path.join(pasta, arquivo)
        try:
            with open(caminho, "r", encoding="utf-8") as f:
                dados = json.load(f)
                arquivos_validos += 1

                if isinstance(dados, list):
                    textos.extend(item.get('text', '') for item in dados if item.get('text'))
                elif isinstance(dados, dict):
                    texto = dados.get('text', '')
                    if texto:
                        textos.append(texto)

                textos_encontrados += len(textos)

        except json.JSONDecodeError:
            print(f"Erro ao ler o arquivo: {arquivo}")

    corpus = ' '.join(textos)
    return corpus, textos, arquivos_validos, textos_encontrados

# Carrega o corpus e os textos
corpus, textos, arquivos_validos, textos_encontrados = carregar_textos_json(pasta_corpus)

# Exibe resultados
print(f"Número de arquivos JSON válidos lidos: {arquivos_validos}")
print(f"Corpus carregado com {len(corpus)} caracteres.")
print(f"Número de textos individuais: {textos_encontrados}")

# Adiciona o caminho do diretório onde está o bpe_tokenizer.py
sys.path.append(r'C:\Users\tagsa\Downloads')
print("Treinando o tokenizador BPE...")

# Define o tamanho do vocabulário e o número de merges
vocab_size = 276
num_merges = vocab_size - 256

# Instancia o tokenizador
tokenizer = BPE_Tokenizer()
tokenizer.train(corpus, vocab_size)

# Lista inicial de tokens codificados
print("Listando tokens...")
tokens = tokenizer.encode(corpus)
ids = list(tokens)  # Converte para lista de IDs

# Dicionário para armazenar os merges realizados
merges = {}

# Função auxiliar para decodificar um par de tokens
def decodificar_par(pair, tokenizer):
    # Decodifica cada token individualmente
    token_a = tokenizer.decode([pair[0]])
    token_b = tokenizer.decode([pair[1]])
    return token_a, token_b

# Realiza os merges e exibe a frequência de cada novo token criado
for i in range(num_merges):
    # Obtém as estatísticas dos pares mais frequentes
    stats = tokenizer.get_stats(ids)
    pair = max(stats, key=stats.get)  # Par mais frequente
    freq = stats[pair]  # Frequência do par
    idx = 256 + i  # Índice do novo token

    # Decodifica os tokens do par para exibir seus caracteres
    token_a, token_b = decodificar_par(pair, tokenizer)

    # Exibe o par sendo mesclado, o novo token, sua frequência e os caracteres
    print(f"Mesclando {pair} ({token_a}, {token_b}) em um novo token {idx} com frequência {freq}")

    # Realiza o merge e armazena no dicionário de merges
    ids = tokenizer.merge(ids, pair, idx)
    merges[pair] = (idx, freq)  # Armazena o índice e a frequência

# Exibe o comprimento dos tokens e IDs após o processo
print("\nComprimento final dos tokens:", len(tokens))
print("Comprimento final dos IDs:", len(ids))

# Calcula e exibe a taxa de compressão
taxa_compressao = len(tokens) / len(ids)
print(f"Taxa de compressão: {taxa_compressao:.2f}X")

Número de arquivos JSON encontrados: 10000


Carregando arquivos JSON: 100%|████████████████████████████████████████████| 10000/10000 [00:16<00:00, 600.79arquivo/s]


Número de arquivos JSON válidos lidos: 10000
Corpus carregado com 71033256 caracteres.
Número de textos individuais: 49977754
Treinando o tokenizador BPE...
Iteração 0/20 em andamento.
Iteração 10/20 em andamento.
Treinamento do tokenizador BPE concluído!
Listando tokens...
Mesclando (195, 163) (�, �) em um novo token 256 com frequência 378704
Mesclando (97, 100) (a, d) em um novo token 257 com frequência 325641
Mesclando (262, 116) (en, t) em um novo token 258 com frequência 322247
Mesclando (256, 256) (o , o ) em um novo token 259 com frequência 320272
Mesclando (195, 167) (�, �) em um novo token 260 com frequência 309541
Mesclando (114, 105) (r, i) em um novo token 261 com frequência 308050
Mesclando (99, 105) (c, i) em um novo token 262 com frequência 304864
Mesclando (97, 260) (a, , ) em um novo token 263 com frequência 294469
Mesclando (114, 101) (r, e) em um novo token 264 com frequência 294374
Mesclando (113, 117) (q, u) em um novo token 265 com frequência 290190
Mesclando (115